#Bronze Layer
1. read data from adls from file & convert into delta table under bronze schema Container = databrickssampledata


In [0]:
dbutils.fs.ls("abfss://kush@adlstrd2025.dfs.core.windows.net/")

[FileInfo(path='abfss://kush@adlstrd2025.dfs.core.windows.net/customers.csv', name='customers.csv', size=231, modificationTime=1752569594000),
 FileInfo(path='abfss://kush@adlstrd2025.dfs.core.windows.net/sales_data.csv', name='sales_data.csv', size=523, modificationTime=1752569594000)]

In [0]:
df1 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://kush@adlstrd2025.dfs.core.windows.net/customers.csv")

df1.display()

customer_id,customer_name,state
C101,Anita,Delhi
C102,Ravi,Maharashtra
C103,Sneha,Karnataka
C104,Amit,Delhi
C105,Neha,Telangana
C106,Kiran,Gujarat
C107,Sana,Karnataka
C108,Vishal,Delhi
C109,Deepa,Tamil Nadu
C110,Rajesh,Maharashtra


In [0]:
df2 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://kush@adlstrd2025.dfs.core.windows.net/sales_data.csv")

df2.display()

transaction_id,customer_id,product_category,amount,transaction_time
T1001,C101,Electronics,550.0,2025-07-14T10:30:00Z
T1002,C102,Fashion,120.5,2025-07-14T11:00:00Z
T1003,C103,Fashion,null,2025-07-14T11:15:00Z
T1004,C104,Grocery,75.0,2025-07-14T12:00:00Z
T1005,C105,Electronics,300.0,2025-07-14T12:30:00Z
T1006,C106,Fashion,150.0,2025-07-14T13:00:00Z
T1007,C107,Fashion,190.0,2025-07-14T13:30:00Z
T1008,C108,Electronics,700.0,2025-07-14T14:00:00Z
T1009,C109,Grocery,55.0,2025-07-14T14:30:00Z
T1010,C110,Fashion,200.0,2025-07-14T15:00:00Z


In [0]:
df1.write.mode("overwrite").saveAsTable("kush_catalog.bronze.customers")
df2.write.mode("overwrite").saveAsTable("kush_catalog.bronze.sales")

#Silver Layer
- Read Bronze tables
- Clean sales data: remove nulls, cast, timestamp,
- Join with customer info
- Write to Silver

In [0]:
customers_bronze = spark.read.table("kush_catalog.bronze.customers")
customers_bronze.display()
sales_bronze = spark.read.table("kush_catalog.bronze.sales")
sales_bronze.display()

customer_id,customer_name,state
C101,Anita,Delhi
C102,Ravi,Maharashtra
C103,Sneha,Karnataka
C104,Amit,Delhi
C105,Neha,Telangana
C106,Kiran,Gujarat
C107,Sana,Karnataka
C108,Vishal,Delhi
C109,Deepa,Tamil Nadu
C110,Rajesh,Maharashtra


transaction_id,customer_id,product_category,amount,transaction_time
T1001,C101,Electronics,550.0,2025-07-14T10:30:00Z
T1002,C102,Fashion,120.5,2025-07-14T11:00:00Z
T1003,C103,Fashion,null,2025-07-14T11:15:00Z
T1004,C104,Grocery,75.0,2025-07-14T12:00:00Z
T1005,C105,Electronics,300.0,2025-07-14T12:30:00Z
T1006,C106,Fashion,150.0,2025-07-14T13:00:00Z
T1007,C107,Fashion,190.0,2025-07-14T13:30:00Z
T1008,C108,Electronics,700.0,2025-07-14T14:00:00Z
T1009,C109,Grocery,55.0,2025-07-14T14:30:00Z
T1010,C110,Fashion,200.0,2025-07-14T15:00:00Z


In [0]:
sales_bronze_cleaned = sales_bronze.na.drop()
sales_bronze_cleaned.display()

transaction_id,customer_id,product_category,amount,transaction_time
T1001,C101,Electronics,550.0,2025-07-14T10:30:00Z
T1002,C102,Fashion,120.5,2025-07-14T11:00:00Z
T1004,C104,Grocery,75.0,2025-07-14T12:00:00Z
T1005,C105,Electronics,300.0,2025-07-14T12:30:00Z
T1006,C106,Fashion,150.0,2025-07-14T13:00:00Z
T1007,C107,Fashion,190.0,2025-07-14T13:30:00Z
T1008,C108,Electronics,700.0,2025-07-14T14:00:00Z
T1009,C109,Grocery,55.0,2025-07-14T14:30:00Z
T1010,C110,Fashion,200.0,2025-07-14T15:00:00Z


In [0]:
sales_customers_joined = sales_bronze_cleaned.join(customers_bronze, on="customer_id", how="inner")
sales_customers_joined.display()

customer_id,transaction_id,product_category,amount,transaction_time,customer_name,state
C101,T1001,Electronics,550.0,2025-07-14T10:30:00Z,Anita,Delhi
C102,T1002,Fashion,120.5,2025-07-14T11:00:00Z,Ravi,Maharashtra
C104,T1004,Grocery,75.0,2025-07-14T12:00:00Z,Amit,Delhi
C105,T1005,Electronics,300.0,2025-07-14T12:30:00Z,Neha,Telangana
C106,T1006,Fashion,150.0,2025-07-14T13:00:00Z,Kiran,Gujarat
C107,T1007,Fashion,190.0,2025-07-14T13:30:00Z,Sana,Karnataka
C108,T1008,Electronics,700.0,2025-07-14T14:00:00Z,Vishal,Delhi
C109,T1009,Grocery,55.0,2025-07-14T14:30:00Z,Deepa,Tamil Nadu
C110,T1010,Fashion,200.0,2025-07-14T15:00:00Z,Rajesh,Maharashtra


In [0]:
sales_customers_joined.write.mode("overwrite").saveAsTable("kush_catalog.silver.sales_customers_joined")


#Gold Layer
- Read Silver Table
- Revenue by state and category
- Write to Gold

In [0]:
analysis_table = spark.read.table("kush_catalog.silver.sales_customers_joined")
analysis_table.display()


customer_id,transaction_id,product_category,amount,transaction_time,customer_name,state
C101,T1001,Electronics,550.0,2025-07-14T10:30:00Z,Anita,Delhi
C102,T1002,Fashion,120.5,2025-07-14T11:00:00Z,Ravi,Maharashtra
C104,T1004,Grocery,75.0,2025-07-14T12:00:00Z,Amit,Delhi
C105,T1005,Electronics,300.0,2025-07-14T12:30:00Z,Neha,Telangana
C106,T1006,Fashion,150.0,2025-07-14T13:00:00Z,Kiran,Gujarat
C107,T1007,Fashion,190.0,2025-07-14T13:30:00Z,Sana,Karnataka
C108,T1008,Electronics,700.0,2025-07-14T14:00:00Z,Vishal,Delhi
C109,T1009,Grocery,55.0,2025-07-14T14:30:00Z,Deepa,Tamil Nadu
C110,T1010,Fashion,200.0,2025-07-14T15:00:00Z,Rajesh,Maharashtra


In [0]:
revenue_by_state = analysis_table.groupBy("state").sum("amount")
revenue_by_state = revenue_by_state.withColumnRenamed("sum(amount)", "revenue")
revenue_by_state.display()


state,revenue
Karnataka,190.0
Tamil Nadu,55.0
Gujarat,150.0
Delhi,1325.0
Maharashtra,320.5
Telangana,300.0


In [0]:
revenue_by_prodcat = analysis_table.groupBy("product_category").sum("amount")
revenue_by_prodcat = revenue_by_prodcat.withColumnRenamed("sum(amount)", "revenue")
revenue_by_prodcat.display() 


product_category,revenue
Fashion,660.5
Grocery,130.0
Electronics,1550.0


In [0]:
revenue_by_state.write.mode("overwrite").saveAsTable("kush_catalog.gold.revenue_by_state")
revenue_by_prodcat.write.mode("overwrite").saveAsTable("kush_catalog.gold.revenue_by_prodcat")